In [1]:
import pandas as pd
import numpy as np
import requests as rq
from io import BytesIO
import unicodedata
import basedosdados as bd
from zipfile import ZipFile
from urllib.request import urlopen
import time as tm
import re 
import os
import glob

In [ ]:
def download_and_unzip(url, extract_to= r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_bcb_estban\estban_agencia'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

#criar uma iteracao de anos e meses 
#usar try except e um break para nao depender de alteracoes nas divs do codigo

#data de inicio 198810
#Atualização mensal
#always n+1 in range
for ano in range(1989,2023):
    for mes in range(1,10):
        #ESTBAN data starts at YEAR=1989 MONTH=10 and currently (2023/01/19) ends at YEAR=2022 MONTH=10
        try:
            #theres a 0 before the numers in 1:9
            estban_file_url = f'https://www4.bcb.gov.br/fis/cosif/cont/estban/agencia/{ano}0{mes}_ESTBAN_AG.ZIP'
            tm.sleep(5)
            download_and_unzip(estban_file_url)
            print(f'The dataset ESTBAN_{ano}0{mes}, was downloaded.')
        except:
            print(f'The dataset ESTBAN_{ano}0{mes}, was not found. Beware, it may not exist.')
    
    for mes in range(10,13):
        #ESTBAN data starts at YEAR=1989 MONTH=10 and currently (2023/01/19) ends at YEAR=2022 MONTH=10
        try:
            estban_file_url = f'https://www4.bcb.gov.br/fis/cosif/cont/estban/agencia/{ano}{mes}_ESTBAN_AG.ZIP'
            tm.sleep(5)
            download_and_unzip(estban_file_url)
            print(f'The dataset ESTBAN_{ano}{mes}, was downloaded.')
        except:
            print(f'The dataset ESTBAN_{ano}{mes}, was not found. Beware, it may not exist.')        
#the first dataset is 1988-10, so gotta make try except 
#it solves all the probs
#if every month the hole code will be rerun, its ok. Otherwise 
#gotta see the releases dates and build a function to get today or thismonth minus -1 to feed range inputs

In [2]:
#definir o que sera usado (funcoes)

#1. dicionario de verbetes
#os nomes do verbete foram retirados do documento  https://www3.bcb.gov.br/aplica/cosif indicado pelo bacen
#foram copiados de forma integral, somente trasnformando de caixa alta para caixa baixa e retirando acentos. 
def dicionario_verbetes():
    verbetes = {#no sitio do banco central o verbete 110 é descrito como 'disponibilidades',
                #já na tabela baixada é descrito como 'encaixe'. 
                '110' : 'disponibilidades',
                '111' : 'caixa',  
                '112' : 'depositos bancarios', 
                '113' : 'banco central - reservas livres em especie',
                '114' : 'aplicacoes temporarias em ouro',
                '120' : 'aplicacoes interfinanceiras de liquidez',
                '130' : 'titulos e valores mobiliarios e instrumentos financeiros derivativos',
                '140' : 'relacoes interfinanceiras e interdependencias',
                '141142' : 'correspondentes no exterior | correspondentes no pais',
                #verbete 143 não existe no site oficial de documentacao do cosif do bacen, mas vem nas tabelas.
                #o nome sera retirado das tabelas
                '143' : 'depend matriz e congeneres no exterior em', 
                '144145146147152' : 'cobranca propria em transito | ordens de pagamento | valores de ligadas e de terceiros em transito | dependencias no pais | suprimentos interdependencias', 
                #mesmo caso do verbete 143, problema e solucao adotada.  
                '153' : 'relacoes com a carteria de cambio e de desenvol',
                '158' : 'outras relacoes interfinanceiras e interdependencias',
                '160' : 'operacoes de credito',
                '161' : 'emprestimos e titulos descontados', 
                '162' : 'financiamentos',
                '163' : 'financiamentos rurais a agricultura (custeio/investimento)',
                '164' : 'financiamentos rurais a pecuaria (custeio/investimento)',
                '165' : 'financiamentos rurais a agricultura (comercializacao)',
                '166' : 'financiamentos rurais a pecuaria (comercializacao)', 
                '167168' : 'financiamentos agroindustriais | renda a apropriar de operacoes de financiamentos rurais e agroindustrias',
                '169' : 'finaciamentos imobiliarios',
                '171' : 'outras operacoes de credito',
                '172' : 'outros creditos',
                '173' : 'creditos em liquidacao',
                '174' : 'provisao para operacoes de credito',
                '176' : 'operacoes especiais',
                '180' : 'arrendamento mercantil',
                '184' : 'provisao para operacoes de arrendamento mercantil',
                '190' : 'outros valores e bens',
                '200' : 'permanente', 
                ##mesmo caso do verbete 143, problema e solucao adotada.  
                '300' : 'contas de compesacao', 
                '399' : 'total do ativo',
                #o nome desses verbetes foi copiado de modo integral de acordo com a planilha do cosif 
                #necessario olhar a discriminacao do cosif para mais contexto 
                '401402403404411412413414415416417418419' : 'servicos publicos | atividades empresariais | especiais do tesouro nacional | saldos credores em contas de emprestimo e financiamento (cpf) | de pessoas fisicas | de pessoas juridicas | de instituicoes do sistema financeiro | judiciais | obrigatorios | para investimentos | vinculados | demais depositos | saldos credores de emprestimos e financiamentos (outros)',
                '420' : 'depositos de poupanca',
                #ambos tem mesmo nome. Ao que tudo indica, o verbeto 430 é a categoria agregada de depositos interfinanceiros
                #o verbete 431 é um componente do 430, assim como os verbetes 431, 432,433
                '430' : 'depositos interfinanceiros',
                '431' : 'depositos interfinanceiros',
                '432' : 'depositos a prazo',
                '433' : 'captacoes no mercado aberto',  
                '440' : 'relacoes interfinanceiras e interdependencias',
                '441442' : 'correspondentes no exterior | correspondentes no pais',
                ##mesmo caso do verbete 143, quanto ao problema e solucao adotada.
                '443' : 'dependencias matriz e congeres no exterior',
                '444445446447456458' : 'cobranca  propria em transito | ordem de pagamento | valores de ligadas e de terceiros em transito | suprimentos interdepartamentais | outras relacoes interfinanceiras e interdependencias',
                ##mesmo caso do verbete 143, quanto ao problema e solucao adotada.
                '457' : 'relacoes com a carteira de cambio e de desenvol',
                '460' : 'obrigacoes por emprestimos e repasses',
                '461462463467468' : 'banco central (assitencia financeira) | redescontos | refinanciamentos | operacoes especiais | outras obrigacoes por emprestimos e repasses',
                '470' : 'instrumentos financeiros derivativos',
                '480' : 'obrigacoes por recebimento',
                '481482483484485487' : 'iof | contribuicoes previdenciarias | tributos estaduais e municipais | tributos federais | fgts | demais recebimentos',
                ##mesmo caso do verbete 143, quanto ao problema e solucao adotada.
                '486' : 'pis/pasep',
                '490500' : 'cheques administrativos | outras obrigacoes',
                '610' : 'patrimonio liquido',
                '710' : 'contas de resultado',
                '711' : 'contas credoras',
                '712' : 'contas devedoras',
                ##mesmo caso do verbete 143, quanto ao problema e solucao adotada. 
                '800' : 'contas de compensacao', 
                '899' : 'total do passivo'
                }
    return verbetes        

#2. atualizar unidades monetarias
def condicoes(database, valor):
    #cruzado
    if database <= 198812:
        return round(valor/(1000**2 * 2750), 6)
    #cruzado novo
    elif database >= 198901 and database <= 199002:
        return round(valor/(1000 * 2750),4)
    #cruzeiro
    elif database >= 199003 and database <= 199307:
        return round(valor/(1000 * 2750),4)
    #cruzeiro real    
    elif database > 199307 and database <= 199406:
        return round(valor/(2750),2)
    #real    
    else: 
        return round(valor,0)

#moedas de acordo com data base
#- Real (R$) : a partir de 07/1994
#    - Cruzeiro Real (CR$) : de 08/1993 a 06/1994 #  / 2750
#    - Cruzeiro (Cr$) : de 03/1990 a 07/1993 #1 / (1000 * 2750)
#    - Cruzado Novo (NCz$) : de 01/1989 a 02/1990 # 1 / (1000 * 2750)
#    - Cruzado (Cz$) : até 12/1988# 1 / (1000^2 * 2750) 


#3. ler arquivos
def ler_arquivos(file)-> pd:
    df = pd.read_csv(file, index_col=None, skiprows= 2, encoding='latin1', sep = ';', 
    #setar o tipo do cnpj como string para ler 0 a esquerda. 
    dtype={ 'CNPJ': 'string',
         'CODMUN': 'string'})
    return df

#4. renomear colunas
def renomear_colunas():
    novos_nomes = {'#DATA_BASE' : 'data_base',
                   'UF' : 'sigla_uf',
                   'CNPJ' : 'cnpj_basico',
                   'NOME_INSTITUICAO' : 'instituicao',
                   'AGENCIA' : 'cnpj_agencia'}
    return novos_nomes

#5.ordenar colunas
def ordenar_colunas():
    ordem  = ['id_municipio',
              'cnpj_basico',
              'instituicao',
              'cnpj_agencia',
              'id_verbete',
              'valor']
    return ordem      

#6. ufs 
def ufs():
  ufs = ['RJ', 'SP', 'ES', 'MG', 'PR', 'SC', 'RS', 'MS', 'GO', 'AC', 'AL', 'AP',
         'AM', 'BA', 'CE', 'DF', 'MA', 'MT', 'PA', 'PB', 'PE', 'PI', 'RN', 'RO',
         'RR', 'SE', 'TO']
  return ufs        

#7 Remover acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])


#8. 
municipios = bd.read_table(dataset_id= 'br_bd_diretorios_brasil',
                           table_id= 'municipio',
                           billing_project_id= "pisagab-staging")

municipios_dict = dict(zip(municipios.id_municipio_bcb, municipios.id_municipio))

Downloading: 100%|██████████| 5570/5570 [00:02<00:00, 2115.26rows/s]


In [ ]:
path = r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_bcb_estban\estban_agencia' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))
dir_path =  r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_bcb_estban\output'


#note que os arquivos sao liberados por meses 
for filename in all_files:
    try:
        #lê arquivo
        df = ler_arquivos(filename)
        print('arquivo lido')

        #tirar acentos dos nomes dos bancos
        df['NOME_INSTITUICAO'] = df['NOME_INSTITUICAO'].apply(remove_accents)
        print('acentos retirados')


        #criar id_municipio de 7 digitos do ibge
        df['id_municipio'] = df.CODMUN.map(municipios_dict)
        print('id_municipio inserido')
        
        #dropa colunas 
        df = df.drop(columns= {'MUNICIPIO', 'CODMUN_IBGE', 'CODMUN'}, axis = 1)     
        print('colunas dropadas')
        
        #renomeia colunas
        df = df.rename(columns = renomear_colunas())
        print('colunas renomeadas')

        #retirar ' antes do cnpj na coluna de agencias
        padrao = re.compile(r"(')")
        df['cnpj_agencia'] = [padrao.sub('',x) for x in df['cnpj_agencia']]
        print("replace da ' feito")

        #converte de wide para long
        df = df.melt(id_vars= ['data_base', 'sigla_uf', 'cnpj_basico', 'instituicao','cnpj_agencia', 'id_municipio'],
            var_name='verbete_descricao', 
            value_name='valor')
        print('de wide para long!')    

        #corrige unidades monetárias
        database = df.data_base
        valor =  df.valor
        condicoes_vetorizada = np.vectorize(condicoes)
        df['valor'] = condicoes_vetorizada(database,valor) #trocar para valor


        #extrai os ids dos verbetes 
        #df['verbete_descricao'].str.extractall(r'([0-9]+)').unstack().fillna('').sum(axis=1).astype(str)
        #otimizar e corrigir a extracao de numeros com regex
        #otimizado
        padrao_letras = re.compile(r'\D')
        df['id_verbete'] = [padrao_letras.sub('',x) for x in df['verbete_descricao']]

        #criar ano e mes 
        df['ano'] = df.data_base.astype(str).str.slice(0,4)
        df['mes'] = df.data_base.astype(str).str.slice(4)


        #a ano e mes das bases sera sempre o mesmo 
        #logo  
        ano = df['ano'][1] 
        mes = df['mes'][1]
        
        for uf in ufs(): 
            uf = uf
            #filtrar uf 
            df2 = df.loc[df['sigla_uf'] == uf]

            # ordenar colunas e remover colunas de particao ano e mes
            #como na funcao ordenar_counas nao tem as vars ano,mes,sigla_uf,database,verbete_descricao elas sao dropadas
            df2 = df2[ordenar_colunas()]

            #criar diretorio para a base
            #chekar path
            particao = dir_path + f'/agencia/ano={ano}/mes={mes}/sigla_uf={uf}'
            if not os.path.exists(particao):
                os.makedirs(particao)
            print(f'diretorio ano_{ano}-mes_{mes}-uf_{uf}')
        
            #salvar df
            particao = dir_path + f'/agencia/ano={ano}/mes={mes}/sigla_uf={uf}/agencia.csv'
            df2.to_csv(particao, index=False, encoding='utf-8', na_rep='')
            print(f'df ano_{ano}-mes_{mes}-uf_{uf}')
    
    except Exception as e:
                erro = []
                erro.append(f"Erro ano{ano}-mes{mes}uf_{uf}")
                print(f"Erro ano{ano}-mes{mes}uf_{uf}")
                print('erro: ', e) 
            